In [2]:
import csv
import sys

# Increase the field size limit (e.g., to 10 MB)
csv.field_size_limit(sys.maxsize)

def tsv_to_csv(input_tsv_file, output_csv_file):
    with open(input_tsv_file, "r", newline='', encoding='utf-8') as tsv_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        
        with open(output_csv_file, "w", newline='', encoding='utf-8') as csv_file:
            csv_writer = csv.writer(csv_file)
            csv_writer.writerows(tsv_reader)

if __name__ == "__main__":
    input_tsv_file = "input_file.tsv"
    output_csv_file = "output_file.csv"

    tsv_to_csv(input_tsv_file, output_csv_file)


OverflowError: Python int too large to convert to C long

In [3]:
import csv
import os
import psycopg2
from psycopg2 import Error

# Define PostgreSQL connection parameters
DB_HOST = "localhost"
DB_PORT = "5432"
DB_NAME = "IDMP"
DB_USER = "postgres"
DB_PASSWORD = "Rsj"

# Define file paths
TSV_FOLDER = "c:/tsv"
TSV_FILE = os.path.join(TSV_FOLDER, "data.tsv")

PSQL_FOLDER = "c:/psql"
SQL_FILE = os.path.join(PSQL_FOLDER, "output_filename.sql")

# Create the PSQL folder if it doesn't exist
if not os.path.exists(PSQL_FOLDER):
    os.makedirs(PSQL_FOLDER)

# Define the SQL INSERT statement template based on the number of columns in the TSV file
INSERT_TEMPLATE = "INSERT INTO your_table_name (column1, column2, ...) VALUES ({0});\n"

def generate_sql_file(tsv_file, sql_file):
    with open(tsv_file, "r", encoding='utf-8') as tsv_file, open(sql_file, "w", encoding='utf-8') as sql_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        headers = next(tsv_reader)  # Get the header row

        # Format the SQL INSERT statement with column names
        insert_statement = INSERT_TEMPLATE.format(", ".join(headers))

        # Write the INSERT statement to the SQL file
        sql_file.write(insert_statement)

        for row in tsv_reader:
            # Create a comma-separated list of values for the current row
            values = ", ".join(f"'{value}'" for value in row)

            # Format the INSERT statement and write it to the SQL file
            insert_statement = INSERT_TEMPLATE.format(values)
            sql_file.write(insert_statement)

    print(f"SQL file generated: {SQL_FILE}")

def import_to_postgresql():
    try:
        # Connect to the PostgreSQL database
        connection = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )

        cursor = connection.cursor()

        # Execute the SQL file using the COPY command to import the TSV data
        with open(SQL_FILE, "r", encoding='utf-8') as sql_file:
            cursor.copy_expert(sql_file.read(), 'your_table_name')

        connection.commit()
        print("Data imported into PostgreSQL successfully!")

    except Error as e:
        print("Error while importing data to PostgreSQL:", e)

    finally:
        if connection:
            cursor.close()
            connection.close()

if __name__ == '__main__':
    # Generate the SQL file
    generate_sql_file(TSV_FILE, SQL_FILE)

    # Import data into PostgreSQL
    import_to_postgresql()


FileNotFoundError: [Errno 2] No such file or directory: 'c:/tsv\\data.tsv'

In [8]:
import psycopg2
import csv
from io import StringIO

def create_table_from_tsv(tsv_file, db_host, db_port, db_name, db_user, db_password, table_name):
    try:
        # Connect to the PostgreSQL database
        connection = psycopg2.connect(
            host=db_host,
            port=db_port,
            database=db_name,
            user=db_user,
            password=db_password
        )

        cursor = connection.cursor()

        # Read the TSV file and extract the column headers
        with open(tsv_file, 'r', encoding='utf-8') as f:
            tsv_reader = csv.reader(f, delimiter='\t')
            headers = next(tsv_reader)

        # Create the table with column headers and define columns as nullable
        columns = ", ".join(f"{header} TEXT" for header in headers)
        create_table_query = f"CREATE TABLE {table_name} ({columns});"
        cursor.execute(create_table_query)

        # Use the COPY command to load data from the TSV file into the table
        with open(tsv_file, 'r', encoding='utf-8') as f:
            # Skip the header row
            next(f)

            # Create an in-memory file-like object to hold the data
            data = StringIO(f.read())

        # Prepare the rows with NULL values for missing fields
        cleaned_data = StringIO()
        for line in data:
            values = line.strip().split('\t')
            cleaned_values = [value if value != '' else 'NULL' for value in values]
            cleaned_data.write('\t'.join(cleaned_values) + '\n')

        cleaned_data.seek(0)  # Reset the buffer position to the beginning

        # Copy the data from the in-memory file-like object into the table
        copy_query = f"COPY {table_name} ({', '.join(headers)}) FROM STDIN DELIMITER E'\\t' CSV NULL 'NULL' HEADER;"
        cursor.copy_expert(copy_query, cleaned_data)

        # Commit the changes
        connection.commit()

        print("Table created and data imported into PostgreSQL successfully!")

    except psycopg2.Error as e:
        print("Error while creating table and importing data to PostgreSQL:", e)

    finally:
        if connection:
            cursor.close()
            connection.close()


if __name__ == "__main__":
    # Replace these values with your PostgreSQL connection details and TSV file path
    db_host = "localhost"
    db_port = "5432"
    db_name = "idmb"
    db_user = "postgres"
    db_password = "Rsj"
    tsv_file = "C:/Users/Rajes/Desktop/PSQL/data.tsv"
    table_name = "title_basics"

    create_table_from_tsv(tsv_file, db_host, db_port, db_name, db_user, db_password, table_name)


Error while creating table and importing data to PostgreSQL: missing data for column "genres"
CONTEXT:  COPY title_basics, line 678273: "tt0701219	tvEpisode	Summer of 4'2"	Summer of 4'2"	0	1996	\N	30	Animation,Comedy"



# Working Codes

**PostgreSQL**

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
import psycopg2


# MySQL database details
mysql_username = 'root'
mysql_password = 'password'
mysql_host = 'localhost'
mysql_port = '3306'
mysql_database = 'gdb041'

# PostgreSQL database details
postgresql_username = 'pg_database_owner'
postgresql_password = 'Rsj'
postgresql_host = 'localhost'
postgresql_port = '5432'
postgresql_database = 'gdb041'

# Connect to the MySQL database using SQLAlchemy
mysql_url = f"mysql://{mysql_username}:{mysql_password}@{mysql_host}:{mysql_port}/{mysql_database}"
mysql_engine = create_engine(mysql_url)

# Connect to the PostgreSQL database using SQLAlchemy
postgresql_url = f"postgresql://{postgresql_username}:{postgresql_password}@{postgresql_host}:{postgresql_port}/{postgresql_database}"
postgresql_engine = create_engine(postgresql_url)

# Table to be migrated
table_name = 'dim_customer'

# Export data from MySQL table to pandas DataFrame
query = f"SELECT * FROM {table_name};"
df = pd.read_sql(query, mysql_engine)

# Create the table in PostgreSQL (using pandas)
df.to_sql(table_name, postgresql_engine, if_exists='replace', index=False, method='multi')

# Optionally, you may need to handle data transformations, indexes, and constraints
# between MySQL and PostgreSQL databases based on the specific schema of the table.

print(f"Migration of table {table_name} from MySQL to PostgreSQL completed successfully.")


In [ ]:
import csv
import os

# Define file paths
TSV_FOLDER = "c:/tsv"
TSV_FILE = os.path.join(TSV_FOLDER, "uncompressed_filename.tsv")

PSQL_FOLDER = "c:/psql"
SQL_FILE = os.path.join(PSQL_FOLDER, "output_filename.sql")

# Create the PSQL folder if it doesn't exist
if not os.path.exists(PSQL_FOLDER):
    os.makedirs(PSQL_FOLDER)

# Define the SQL INSERT statement template based on the number of columns in the TSV file
INSERT_TEMPLATE = "name_basics (column1, column2, ...) VALUES ({0});\n"

with open(TSV_FILE, "r", encoding='utf-8') as tsv_file, open(SQL_FILE, "w", encoding='utf-8') as sql_file:
    tsv_reader = csv.reader(tsv_file, delimiter="\t")
    next(tsv_reader)  # Skip header if present

    for row in tsv_reader:
        # Create a comma-separated list of values for the current row
        values = ", ".join(f"'{value}'" for value in row)

        # Format the INSERT statement and write it to the SQL file
        insert_statement = INSERT_TEMPLATE.format(values)
        sql_file.write(insert_statement)

print(f"SQL file generated: {SQL_FILE}")


In [ ]:
import csv
import os
import psycopg2
from psycopg2 import Error

# Define PostgreSQL connection parameters
DB_HOST = "your_host"
DB_PORT = "your_port"
DB_NAME = "your_database_name"
DB_USER = "your_username"
DB_PASSWORD = "your_password"

# Define file paths
TSV_FOLDER = "c:/tsv"
TSV_FILE = os.path.join(TSV_FOLDER, "uncompressed_filename.tsv")

PSQL_FOLDER = "c:/psql"
SQL_FILE = os.path.join(PSQL_FOLDER, "output_filename.sql")

# Create the PSQL folder if it doesn't exist
if not os.path.exists(PSQL_FOLDER):
    os.makedirs(PSQL_FOLDER)

# Define the SQL INSERT statement template based on the number of columns in the TSV file
INSERT_TEMPLATE = "INSERT INTO your_table_name (column1, column2, ...) VALUES ({0});\n"

def generate_sql_file(tsv_file, sql_file):
    with open(tsv_file, "r", encoding='utf-8') as tsv_file, open(sql_file, "w", encoding='utf-8') as sql_file:
        tsv_reader = csv.reader(tsv_file, delimiter="\t")
        headers = next(tsv_reader)  # Get the header row

        # Format the SQL INSERT statement with column names
        insert_statement = INSERT_TEMPLATE.format(", ".join(headers))

        # Write the INSERT statement to the SQL file
        sql_file.write(insert_statement)

        for row in tsv_reader:
            # Create a comma-separated list of values for the current row
            values = ", ".join(f"'{value}'" for value in row)

            # Format the INSERT statement and write it to the SQL file
            insert_statement = INSERT_TEMPLATE.format(values)
            sql_file.write(insert_statement)

    print(f"SQL file generated: {SQL_FILE}")

def import_to_postgresql():
    try:
        # Connect to the PostgreSQL database
        connection = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )

        cursor = connection.cursor()

        # Execute the SQL file using the COPY command to import the TSV data
        with open(SQL_FILE, "r", encoding='utf-8') as sql_file:
            cursor.copy_expert(sql_file.read(), 'your_table_name')

        connection.commit()
        print("Data imported into PostgreSQL successfully!")

    except Error as e:
        print("Error while importing data to PostgreSQL:", e)

    finally:
        if connection:
            cursor.close()
            connection.close()

if __name__ == '__main__':
    # Generate the SQL file
    generate_sql_file(TSV_FILE, SQL_FILE)

    # Import data into PostgreSQL
    import_to_postgresql()
